In [1]:
## import statements ##
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import missingno as
%matplotlib inline

In [2]:
train_data = pd.read_csv('train_tweets.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [4]:
train_data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

#### Cleaning the data

In [5]:
def drop_features(features,data):
    data.drop(features,inplace=True,axis=1)

In [6]:
import re
## example ## 
re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ","ouch...junior is angryð#got7 #junior #yugyo..., @user")

'ouch   junior is angry     got7  junior  yugyo      '

In [7]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

In [8]:
train_data['processed_tweets'] = train_data['tweet'].apply(process_tweet)

In [9]:
train_data.head(10)

,id,label,tweet,processed_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before they ...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champions clev...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it s so gr8


In [10]:
drop_features(['id','tweet'],train_data)

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 2 columns):
label               31962 non-null int64
processed_tweets    31962 non-null object
dtypes: int64(1), object(1)
memory usage: 499.5+ KB


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(train_data["processed_tweets"],train_data["label"], test_size = 0.2, random_state = 42)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [15]:
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [16]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [17]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(25569, 33735)
(25569, 33735)


In [18]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [19]:
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(6393, 33735)
(6393, 33735)


In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
predictions = model.predict(x_test_tfidf)

In [22]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

array([[5905,   32],
       [ 217,  239]])

In [23]:
# tp,fn,fp,tn = c_m[0][0],c_m[0][1],c_m[1][0],c_m[1][1]
# precision = tp/(tp+fp)
# recall = tp/(tp+fn)
# f1score = 2 * (recall * precision) / (recall + precision)
# f1score

In [24]:
f1_score(y_test,predictions)

0.657496561210454

##### Preparing test data

In [25]:
hashtag = '#'+input()

trump


In [26]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = 'm8HzLkLS38IfLjSlUgWyeKLjQ'
consumer_secret = 'QUJbkeXYEUnhL7GVEcFFePTlPire9AzbCHzEvl8cdyBAWP0Mut'
access_token = '282922644-K88LhsZBkgPAH2Ajs2TvVJimOherryCiBzfZrZXG'
access_token_secret = 'LHn23GNpD6jzSE8gS5FRPji4gIzCawN5wBACX1XnFQ0jV'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open(hashtag+'.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
enum = 1
csvWriter.writerow(['id','tweet'])
for tweet in tweepy.Cursor(api.search,q=hashtag,count=100,
                           lang="en",
                           since="2019-09-14").items(500):
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([enum,tweet.text.encode('utf-8')])
    enum+=1

2019-09-16 07:26:56 RT @AndyOstroy: @RadioFreeTom Absolutely. Given that we have a dangerous pathologically-lying sexist racist corrupt sociopath traitor squat…
2019-09-16 07:26:55 RT @mayawiley: #Trump says #DOJ should step in to help #Kavanaugh. THAT is NOT the job of the Justice department. What should have been it’…
2019-09-16 07:26:53 RT @EnglishAlwaght: US ’Locked &amp; Loaded’ for Response after Saudi Oil Strike
https://t.co/V9uOFPSAFk
#US  
#Trump  
#AramcoDroneStrike http…
2019-09-16 07:26:52 RT @AndyOstroy: @TheRickWilson Absolutely. Given that we have a dangerous pathologically-lying sexist racist corrupt sociopath traitor squa…
2019-09-16 07:26:51 RT @confunctionist: #Trump releasing oil from the #StrategicPetroleumReserve will do to renewables (and carbon the energy industry) what he…
2019-09-16 07:26:39 @s76l42 @itsJeffTiedrich @realDonaldTrump Apparently #Trump has deep personal financial interests with Saudi. They… https://t.co/0ihWazpdrw
2019-09-16 07:26:38 #Trump wait

2019-09-16 07:14:18 RT @rahas44: @realDonaldTrump #MAGA 🇺🇸🇺🇸🇺🇸
❌restart opposition is being censored heavily by Media❌They have been attacking patriotic sounds…
2019-09-16 07:14:16 RT @Jews4Trump: From @IsraeliPM to @POTUS @realDonaldTrump:

"Together, we will continue full steam ahead with our common battle against te…
2019-09-16 07:14:11 RT @RN_JB7: Tech Giant determine to stop #Trump’s re-election in 2020 — Exclusive: Google Insider Turns Over 950 Pages Of Docs And Laptop T…
2019-09-16 07:14:08 Top stories of today, you shouldn't miss!

1- Bad days gone as Pakistan moving towards economic stability: Hafeez S… https://t.co/N6Pc17Q5tr
2019-09-16 07:13:45 RT @the_hindu: This is the first time in recent history that the leaders of the two largest democracies would be addressing a joint rally a…
2019-09-16 07:13:40 RT @maryhilliard45: Every decision you make @realDonaldTrump will make your numbers drop.
Add this love relationship with Saudia Arabia wit…
2019-09-16 07:13:33 RT @BuzzlyIn: 

2019-09-16 07:03:57 What’s America coming too? Our foreign policy this weekend is being dictated by #Riyadh, while #Trump  and Democrat… https://t.co/e91eVLBq4h
2019-09-16 07:03:55 #Trump #Trump2020 #MAGA #KAG #POTUS 
Since the latest ATTACKS by #Taliban mean any Talks with them are DEAD, so too… https://t.co/xs2yhJHYP6
2019-09-16 07:03:43 #BREAKING
The president #Trump says the #US is targeted and armed to retaliate Iran missile attacks
My take: this i… https://t.co/pSQCdrbeGV
2019-09-16 07:03:43 RT @Iran_Military: #Trump: #Saudi will export more oil to compensate for sanctioned Iranian #oil.

[A single #Yemeni #drone attack knocks o…
2019-09-16 07:03:34 US "locked and loaded" to respond to attack on Saudi oil: Trump #Politics #Political #Trump https://t.co/3AN2aboHNN
2019-09-16 07:03:32 Iranian Foreign Ministry spokesman:
A meeting between #Rouhani &amp; #Trump in #NY is not on the agenda https://t.co/sgf5sQ5CGD
2019-09-16 07:03:29 Don't dishonore this 2020 election by leaving this o

2019-09-16 06:55:39 RT @carlyinNJ: @CheriJacobus @lauferlaw Always had concerns about George Conway, has he been playing the #Resistance? Just sayin, Kellyanne…
2019-09-16 06:55:30 RT @AndyOstroy: .@brithume...how do you know it's a smear? Nothing's been investigated yet. In prejudging like this, you are 'smearing' the…
2019-09-16 06:55:30 RT @DebraMessing: 37 INDICTMENTS 

10 ACTS OF COLLUSION

COUNTLESS ACTS OF CORRUPTION

Don’t fall for distractions. #IMPEACHMENTHEARINGS ar…
2019-09-16 06:55:27 RT @BabakTaghvaee: #BREAKING: Two hours ago, #Trump administration released Satellite images &amp; proved that No #Houthi drone was involved in…
2019-09-16 06:55:23 RT @naimish9558: After Modi's Invitation, Trump be like
#HowdyModi 
#trump https://t.co/tSWEVRq1nr
2019-09-16 06:55:06 RT @AssangeMrs: This "Q" nonsense about my son Julian is a psop!

Designed to deter people from protesting to protect Julian

The #Trump ad…
2019-09-16 06:55:03 ‘He’s a businessman’: One of Hong Kong’s most promin

2019-09-16 06:45:13 Hitler Germany November 12 1933 - via @smedian_network

.@Medium #medium #blog #essay #politics #news #government… https://t.co/OSa7Df3s5m
2019-09-16 06:45:12 #Trump Tweets Decodes for 9/15/2019 https://t.co/Be3uXjFg2e via @YouTube
2019-09-16 06:45:09 RT @KevinMcshea: Guy who can't even tell the truth about a weather report .... Republican Terrorist Party @realDonaldTrump @seanhannity #Tr…
2019-09-16 06:45:07 RT @AndyOstroy: @realDonaldTrump Shame on you. But not a surprise. One sexual-abuser defending and protecting another... #Trump #KavanaughR…
2019-09-16 06:45:05 RT @Miami_Rebelde: WARNING these two accounts spy on Leftists here. They are also working together to report Left Wing profiles. Please rep…
2019-09-16 06:45:01 #Trump’s decision to withhold military aid to #Ukraine has again drawn attention to the rationality of Ukraine’s wa… https://t.co/0tE3qeqbKl
2019-09-16 06:45:00 #WhiteHouse confirms #Trump's participation at #HowdyModi event https://t.co/1bWinys

In [ ]:
test_data = pd.read_csv(hashtag+'.csv')

In [ ]:
test_data.info()

In [ ]:
test_data['processed_tweet'] = test_data['tweet'].apply(process_tweet)

In [ ]:
test_data.head()

In [ ]:
drop_features(['tweet'],test_data)

In [ ]:
train_counts = count_vect.fit_transform(train_data['processed_tweets'])
test_counts = count_vect.transform(test_data['processed_tweet'])

In [ ]:
print(train_counts.shape)
print(test_counts.shape)

In [ ]:
train_tfidf = transformer.fit_transform(train_counts)
test_tfidf = transformer.transform(test_counts)

In [ ]:
print(train_tfidf.shape)
print(test_tfidf.shape)

In [ ]:
model.fit(train_tfidf,train_data['label'])

In [ ]:
predictions = model.predict(test_tfidf)

In [ ]:
final_result = pd.DataFrame({'id':test_data['id'],'label':predictions})
final_result.to_csv('output.csv',index=False)